In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import plot
from numpy import pi as π
import firedrake
from firedrake import inner, as_vector, sqrt, sin, cos, grad, dx

# The minimal surface equation

In this demo I'll show a nonlinear elliptic PDE, the minimal surface equation.
The minimal surface equation can be best described via a principle for a field $u$.
The *action* functional is the quantity

$$J(u) = \int_\Omega\sqrt{1 + |\nabla u|^2}dx$$

i.e. the area of the graph of $u$.
The goal is to find the minimizer of $J$ subject to the Dirichlet boundary condition

$$u|_{\partial\Omega} = g.$$

The functional derivative of $J$ along a perturbation $v$ is

$$\left\langle dJ(u), v\right\rangle = \int_\Omega\frac{\nabla u\cdot\nabla v}{\sqrt{1 + |\nabla u|^2}}dx$$

You can compute the second derivative but part of what I'll show is how to use the symbolic differentiation capability of firedrake.
We'll then implement a Newton-type method to solve for $u$.

The domain will be the same as before, and for the boundary conditions we'll use the trigonometric polynomial from the Laplace demo.

In [ ]:
mesh = firedrake.Mesh('domain.msh')
Q = firedrake.FunctionSpace(mesh, family='CG', degree=1)

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

a1 = 1.
k1 = as_vector((2, 1))
ϕ1 = π * inner(k1, x)

a2 = 1/4
k2 = as_vector((4, 6))
ϕ2 = π * inner(k2, x)

expr = a1 * sin(ϕ1) + a2 * cos(ϕ2)
g = firedrake.interpolate(expr, Q)

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
contours = plot.tricontourf(g, 40, cmap='viridis', axes=axes)
fig.colorbar(contours)
fig.show()

Forming the action functional works just like any other functional.

In [ ]:
u = g.copy(deepcopy=True)
J = sqrt(1 + inner(grad(u), grad(u))) * dx

In [ ]:
print('Initial surface area: {}'.format(firedrake.assemble(J)))

Firedrake can compute the functional derivative of $J$ with respect to $u$ by using a symbolic algebra package under the hood.

In [ ]:
F = firedrake.derivative(J, u)

We can go one step further and compute the second derivative.
This will come in handy for using Newton's method.

In [ ]:
H = firedrake.derivative(F, u)

As a sanity check, the method `arguments` of a form returns a list of its arguments.
A rank-0 form, i.e. a functional has no arguments, only coefficients.
A rank-1 form has one argument, a test function.
A rank-2 form has two arguments, a test function and a trial function.

In [ ]:
print('Rank of J: {}'.format(len(J.arguments())))
print('Rank of F: {}'.format(len(F.arguments())))
print('Rank of H: {}'.format(len(H.arguments())))

The field $u$ is not an argument, but rather a *coefficient* of $J$, $F$, and $H$.

In [ ]:
u is J.coefficients()[0]

Let's compute the Newton search direction $v$ to get a perturbation to $u$.
Since $u$ is a copy of $g$ and thus already satisfies the boundary conditions, we can set 0 boundary conditions in our solve for $v$.

In [ ]:
v = firedrake.Function(Q)
bc = firedrake.DirichletBC(Q, 0, 'on_boundary')
firedrake.solve(H == -F, v, bc,
                solver_parameters={'ksp_type': 'preonly',
                                   'pc_type': 'lu'})

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
contours = plot.tricontourf(v, 40, cmap='viridis', axes=axes)
fig.colorbar(contours)
fig.show()

We can calculate the derivative of $J$ along $v$ by using the function `action`.
In this case $F$, the derivative of $J$, is a rank-1 form. 
The action of $F$ on $v$ gives a rank-0 form that we can assemble into a scalar.

In [ ]:
directional_derivative = firedrake.action(F, v)

In [ ]:
print('Rank of the directional derivative: {}'
      .format(len(directional_derivative.arguments())))

In [ ]:
print('Value of the directional derivative: {}'
      .format(firedrake.assemble(directional_derivative)))

The action functional $J$ is decreasing along $v$, which is just what you'd want from a search direction.
Now let's try to do a line search along $v$ -- find a value of $t$ such that $J(u + t \cdot v)$ is smaller than $J(u)$.

In [ ]:
t = firedrake.Constant(1)
u_t = u + t * v
J_t = firedrake.replace(J, {u: u_t})
J_0 = firedrake.assemble(J)
print('t            |   δJ')
for k in range(10):
    print('{:4e} | {}'.format(1.0/2**k, firedrake.assemble(J_t) - J_0))
    t.assign(0.5 * t)

In this case, a step length of 1/16 gives an appreciable decrease in surface area.

**Exercise**: Make this into a backtracking Newton search!